In [ ]:
from keras import Input, layers
from keras import Model
import numpy as np

""" Реализация модели с тремя выходами с использованием функц-го API."""
vocabulary_size = 50000
num_income_groups = 10

# Для выходных слоев определены имена
post_input = Input(shape=(None,), dtype = 'int32', name='posts')

embedded_post = layers.Embedding(
    256, vocabulary_size)(post_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_post)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(
    1,
    activation='sigmoid',
    name='gender')(x)

model = Model(post_input,
              [age_prediction, income_prediction, gender_prediction])


model.compile(optimizer='rmsprop',
              loss= ['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

# Параметры компиляции модели с нес вых: взвешивание потерь
model.compile(optimizer='rmsprop',
              loss= {'age': 'mse',
                     'income': 'categorical_crossentropy',
                     'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

print(model.summary())

# Передача данных в модель с несколькими выходами
# Предаолагается, что age_targets, income_targets, gender_targets - это
# массивы Numpy
model.fit(posts,
          [age_targets, income_targets, gender_targets],
          epochs=10,
          batch_size=64)

# Эквивалентное решение - только если определены имена выходных слоев
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)

